In [1]:
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import time
import pprint
from tqdm.notebook import trange

from sac import Agent
from obs import observation_shape
import json

def load_config():
    f = open('env.json',) 
    data = json.load(f) 
    f.close()

    return data

filename = 'inverted_pendulum.png'

best_score = -10000.0
best_score = -1000.0
score_history = []
episode_lens = []
avg_history = []
std_history = []
avg_history_100 = []

2024-02-08 22:31:46.091503: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-08 22:31:46.118798: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-08 22:31:46.118830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-08 22:31:46.119491: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-08 22:31:46.123915: I tensorflow/core/platform/cpu_feature_guar

In [2]:
data = load_config()
env = gym.make('racetrack-v0', render_mode = 'rgb_array')
env.configure(data) # type: ignore
pprint.pprint(env.config) # type: ignore
(obs, info), done = env.reset(), False

#observation config
proc = observation_shape(obs,info,2)
proc.reset()
input = proc.get_input()
print(input.shape)


{'action': {'lateral': True, 'longitudinal': True, 'type': 'ContinuousAction'},
 'action_reward': -0.3,
 'centering_position': [0.5, 0.5],
 'collision_reward': -1,
 'controlled_vehicles': 1,
 'duration': 80,
 'lane_centering_cost': 4,
 'lane_centering_reward': 1,
 'manual_control': False,
 'observation': {'features': ['presence',
                              'x',
                              'y',
                              'vx',
                              'vy',
                              'cos_h',
                              'sin_h',
                              'heading',
                              'long_off',
                              'lat_off',
                              'ang_off'],
                 'features_range': {'vx': [-20, 20],
                                    'vy': [-20, 20],
                                    'x': [-100, 100],
                                    'y': [-100, 100]},
                 'type': 'Kinematics',
                 'vehicles_c

/home/o/Documents/thesis/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(
/home/o/Documents/thesis/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.config to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.config` for environment variables or `env.get_wrapper_attr('config')` that will search the reminding wrappers.
  logger.warn(


In [3]:
agent = Agent(input_dims=input.shape, env=env,
            n_actions=2)

2024-02-08 22:31:50.706393: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-08 22:31:50.730880: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-08 22:31:50.731021: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
# main loop
for episode in trange(500, desc='Test episodes'):

        proc.reset()        
        (observation, info), done = env.reset(), False
        truncated = False
        
        proc.update_input(observation, info)
        observation = proc.get_input()

        episode_reward = 0
        episode_len = 0

        while (not done) and (not truncated):
            action = agent.choose_action(observation)

            new_observation, reward, done, truncated, new_info = env.step(action=action)
            proc.update_input(new_observation, info)
            new_observation = proc.get_input()

            episode_reward += reward # type: ignore
            episode_len +=1

            if new_info["rewards"]["on_road_reward"] == False or episode_len >= 200:
                truncated = True

            agent.remember(state=observation, action=action, done=done,
                            reward=reward, new_state=new_observation)
            agent.learn()

            observation = new_observation

        episode_lens.append(episode_len)

        score_history.append(episode_reward)
        avg_score = np.mean(score_history)
        avg_history.append(avg_score)
        std_score = np.std(score_history)
        std_history.append(std_score)

        avg_score_100 = np.mean(score_history[-100:])
        avg_history_100.append(avg_score_100)

        # agent.save_models(episode)

        agent.tensorboard.update_stats(episode_rew = episode_reward, 
                                       average_rew =avg_score,
                                       average_100_reward = avg_score_100,
                                       std_rew=std_score, 
                                       episode_len = episode_len)

        print('episode ', episode, 'score %.1f' % episode_reward, 'ep len', episode_len,
              'avg score %.1f' % avg_score, 'avg_score_100 %.1f' %avg_score_100,'std score %.1f' % std_score)



Test episodes:   0%|          | 0/500 [00:00<?, ?it/s]

episode  0 score -7.3 ep len 25 avg score -7.3 avg_score_100 -7.3 std score 0.0
episode  1 score -8.5 ep len 26 avg score -7.9 avg_score_100 -7.9 std score 0.6
episode  2 score 0.1 ep len 4 avg score -5.2 avg_score_100 -5.2 std score 3.8
episode  3 score 0.0 ep len 4 avg score -3.9 avg_score_100 -3.9 std score 4.0
episode  4 score -8.8 ep len 26 avg score -4.9 avg_score_100 -4.9 std score 4.1


2024-02-08 22:31:57.782502: I external/local_xla/xla/service/service.cc:168] XLA service 0xfdd0bf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-08 22:31:57.782528: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-02-08 22:31:57.785389: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-08 22:31:57.799534: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1707420717.844345   60756 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


episode  5 score -15.4 ep len 55 avg score -6.6 avg_score_100 -6.6 std score 5.4
episode  6 score -0.2 ep len 5 avg score -5.7 avg_score_100 -5.7 std score 5.5
episode  7 score 0.5 ep len 6 avg score -5.0 avg_score_100 -5.0 std score 5.5
episode  8 score -0.2 ep len 3 avg score -4.4 avg_score_100 -4.4 std score 5.4
episode  9 score 2.6 ep len 11 avg score -3.7 avg_score_100 -3.7 std score 5.5
episode  10 score 0.3 ep len 10 avg score -3.4 avg_score_100 -3.4 std score 5.4
episode  11 score -0.3 ep len 3 avg score -3.1 avg_score_100 -3.1 std score 5.2
episode  12 score 2.0 ep len 11 avg score -2.7 avg_score_100 -2.7 std score 5.2
episode  13 score 0.4 ep len 3 avg score -2.5 avg_score_100 -2.5 std score 5.1
episode  14 score -0.5 ep len 17 avg score -2.4 avg_score_100 -2.4 std score 4.9
episode  15 score -0.0 ep len 20 avg score -2.2 avg_score_100 -2.2 std score 4.8
episode  16 score -0.3 ep len 5 avg score -2.1 avg_score_100 -2.1 std score 4.7
episode  17 score -0.5 ep len 5 avg score -

KeyboardInterrupt: 

In [5]:
n_steps = sum(episode_lens)
print(n_steps)


15164


In [ ]:

# env = record_videos(env)
# from gymnasium.wrappers import RecordVideo

data = load_config()
env = gym.make('racetrack-v0', render_mode = 'human')
env.configure(data) # type: ignore

# env = RecordVideo(env, video_folder="videos", episode_trigger=lambda e: True)
# env.unwrapped.set_record_video_wrapper(env)


for episode in trange(1, desc='Test episodes'):
        env.render()
        proc.reset()        
        (observation, info), done = env.reset(), False
        truncated = False
        
        proc.update_input(observation, info)
        observation = proc.get_input()

        episode_reward = 0
        episode_len = 0

        while (not done) and (not truncated):
            action = agent.choose_action(observation)

            new_observation, reward, done, truncated, new_info = env.step(action=action)
            proc.update_input(new_observation, info)
            new_observation = proc.get_input()

            episode_reward += reward # type: ignore
            episode_len +=1

            if new_info["rewards"]["on_road_reward"] == False or episode_len >= 300:
                truncated = True

            observation = new_observation


env.close()
